<a href="https://colab.research.google.com/github/maggieliuzzi/reinforcement_learning/blob/master/dynamic_programming/control/PolicyIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dynamic Programming** | Control Problem | Policy Iteration

Find optimal policy and value function.

- State Transitions (the next state and reward given your action-state pair): 
  1) deterministic (all p(s',r|s,a) = 1 or 0),
  2) probabilistic (0.5 for desired position, 0.5/3 in any of the other three)


In [1]:
from __future__ import print_function, division
from builtins import range
import numpy as np
!wget "https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/rl/grid_world.py"
from grid_world import standard_grid, negative_grid
!wget "https://raw.githubusercontent.com/maggieliuzzi/reinforcement_learning/master/environments/utils.py"
from utils import print_values, print_policy

--2020-06-02 02:52:36--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/rl/grid_world.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7531 (7.4K) [text/plain]
Saving to: ‘grid_world.py’

grid_world.py       100%[===================>]   7.35K  --.-KB/s    in 0s      

2020-06-02 02:52:36 (84.6 MB/s) - ‘grid_world.py’ saved [7531/7531]

--2020-06-02 02:52:37--  https://raw.githubusercontent.com/maggieliuzzi/reinforcement_learning/master/environments/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [0]:
SMALL_ENOUGH = 1e-3
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

In [3]:
grid = negative_grid()  # reward of -0.1 for every non-terminal state
grid = negative_grid(step_cost=-1.0)  # for probabilistic

print("rewards:")
print_values(grid.rewards, grid)

rewards:
---------------------------
-1.00|-1.00|-1.00| 1.00|
---------------------------
-1.00| 0.00|-1.00|-1.00|
---------------------------
-1.00|-1.00|-1.00|-1.00|


In [4]:
## Uniform random policy
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

print("initial policy:")
print_policy(policy, grid)

initial policy:
---------------------------
  R  |  D  |  D  |     |
---------------------------
  U  |     |  L  |     |
---------------------------
  U  |  D  |  R  |  R  |


In [0]:
V = {}
states = grid.all_states()
for s in states:
  # V[s] = 0
  if s in grid.actions:
    V[s] = np.random.random()
  else:
    # terminal state
    V[s] = 0

In [0]:
# repeat until convergence - will break out when policy does not change
while True:

  # policy evaluation step - we already know how to do this!
  while True:
    biggest_change = 0
    for s in states:
      old_v = V[s]

      ''' 
      ## Deterministic state transitions
      # V(s) only has value if it's not a terminal state
      if s in policy:
        a = policy[s]
        grid.set_state(s)
        r = grid.move(a)
        V[s] = r + GAMMA * V[grid.current_state()]
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))
      '''
      ## Probabilistic state transitions
      new_v = 0
      if s in policy:
        for a in ALL_POSSIBLE_ACTIONS:
          if a == policy[s]:
            p = 0.5
          else:
            p = 0.5/3
          grid.set_state(s)
          r = grid.move(a)
          new_v += p*(r + GAMMA * V[grid.current_state()])
        V[s] = new_v
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))
    
    
    if biggest_change < SMALL_ENOUGH:
      break

  # policy improvement step
  is_policy_converged = True
  for s in states:
    if s in policy:
      old_a = policy[s]
      new_a = None
      best_value = float('-inf')

      '''
      ## Deterministic state transitions
      # loop through all possible actions to find the best current action
      for a in ALL_POSSIBLE_ACTIONS:
        grid.set_state(s)
        r = grid.move(a)
        v = r + GAMMA * V[grid.current_state()]
        if v > best_value:
          best_value = v
          new_a = a
      '''
      ## Probabilistic state transitions
      # loop through all possible actions to find the best current action
      for a in ALL_POSSIBLE_ACTIONS: # chosen action
        v = 0
        for a2 in ALL_POSSIBLE_ACTIONS: # resulting action
          if a == a2:
            p = 0.5
          else:
            p = 0.5/3
          grid.set_state(s)
          r = grid.move(a2)
          v += p*(r + GAMMA * V[grid.current_state()])
        if v > best_value:
          best_value = v
          new_a = a
    

      policy[s] = new_a
      if new_a != old_a:
        is_policy_converged = False

  if is_policy_converged:
    break

In [7]:
print("values:")
print_values(V, grid)
print("policy:")
print_policy(policy, grid)

values:
---------------------------
-4.52|-2.95|-0.86| 0.00|
---------------------------
-5.57| 0.00|-1.94| 0.00|
---------------------------
-5.76|-4.88|-3.44|-2.17|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  R  |  R  |  U  |  U  |
